In [ ]:
import serial
import matplotlib.pyplot as plt
import numpy as np 
import time
from flask import Flask, jsonify, send_from_directory  
import random
from flask_cors import CORS

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})  # Allow all domains to access all routes

# 设置串口参数
port0 = 'COM3'  # 根据你的 Arduino 连接的串口号进行调整
port1 = 'COM4'
baudrate = 9600  # 根据你的 Arduino 设置的波特率进行调整

# 检查串口是否已关闭
try:
    ser0 = serial.Serial(port0, baudrate)
    ser1 = serial.Serial(port1, baudrate)
    ser0.close()
    ser1.close()
except serial.SerialException:
    print("请关闭串口连接后再运行程序！")
    exit()

# 打开串口连接
ser0 = serial.Serial(port0, baudrate)
ser1 = serial.Serial(port1, baudrate)

# 开始计时
start_time = time.time()

# python 运行时间
time_data = []

# 设置数据采集时间
data_collection_time = 200 # 采集时间s

@app.route('/html')
def index():
    return send_from_directory('D:/2024-Projects/4-Sensor for tensegrity/3-Journal/Fig.7 LLM-assited system/1-Monitor/3-Wireless moinitor/', 'index.html')

@app.route('/data')
def get_data():
    # Real-time data reading and processing
    if ser0.in_waiting > 0 and ser1.in_waiting > 0:
        data0 = ser0.readline().decode('utf-8').rstrip()
        data1 = ser1.readline().decode('utf-8').rstrip()
        
        try:
            # Split data received from both serial ports
            data = data0.split(',') + data1.split(',')
            R_B21C12, R_A11C12, R_A11B21, R_B22C12, R_A21B22, R_A21C12, R_A21C11, R_A21B12, R_B12C11, R_A22C22, R_A22B22, R_B22C22, R_A22B12, R_A22C21, R_B12C21, R_A12C21, R_A12B11, R_B11C21, R_A12B21, R_A12C22, R_B21C22, R_A11B11, R_A11C11, R_B11C11 = map(float, data)
            
            # Create a dictionary with the resistance values
            resistance_data = {
                'R_B21C12': R_B21C12, 'R_A11C12': R_A11C12, 'R_A11B21': R_A11B21,
                'R_B22C12': R_B22C12, 'R_A21B22': R_A21B22, 'R_A21C12': R_A21C12,
                'R_A21C11': R_A21C11, 'R_A21B12': R_A21B12, 'R_B12C11': R_B12C11,
                'R_A22C22': R_A22C22, 'R_A22B22': R_A22B22, 'R_B22C22': R_B22C22,
                'R_A22B12': R_A22B12, 'R_A22C21': R_A22C21, 'R_B12C21': R_B12C21,
                'R_A12C21': R_A12C21, 'R_A12B11': R_A12B11, 'R_B11C21': R_B11C21,
                'R_A12B21': R_A12B21, 'R_A12C22': R_A12C22, 'R_B21C22': R_B21C22,
                'R_A11B11': R_A11B11, 'R_A11C11': R_A11C11, 'R_B11C11': R_B11C11
            }
        except ValueError:
            return "Invalid sensor value", 400  # Return HTTP 400 for bad request
        
        return jsonify(resistance_data)

    return "No data available", 404  # Return HTTP 404 if no data is available

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
